In [1]:
!pip install -q language-tool-python --no-index --find-links ../input/daigt-misc/
!mkdir -p /root/.cache/language_tool_python/
!cp -r /kaggle/input/daigt-misc/lang57/LanguageTool-5.7 /root/.cache/language_tool_python/LanguageTool-5.7

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# import cudf as pd
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string
import regex as re
# import nltk
# from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import wordnet
# nltk.download("wordnet")
# nltk.download('punkt')
# nltk.download("omw-1.4")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from scipy.sparse import csr_matrix, vstack, hstack
import language_tool_python
from concurrent.futures import ProcessPoolExecutor
from sklearn.metrics import accuracy_score

In [4]:
# Dataset from the competition
train_essays_base = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
test_essays_base = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
train_prompts_base = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv')

# Dataset from argugpt
argu_base = pd.read_csv('/kaggle/input/argugpt/argugpt.csv')

# Dataset from daigt
# daigt_1_base = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_01.csv')
# daigt_2_base = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_02.csv')
# daigt_3_base = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_03.csv')
# daigt_4_base = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_04.csv')

# Dataset from external daigt
# xdaigt_base = pd.read_csv('/kaggle/input/daigt-external-dataset/daigt_external_dataset.csv')

# Dataset from daigt v2
daigtv2_base = pd.read_csv('/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv')

# Dataset from hello claude
# hello_base = pd.read_csv('/kaggle/input/hello-claude-1000-essays-from-anthropic/persuade15_claude_instant1.csv')

# Dataset from Rdizzl3
llm_base = pd.read_csv('/kaggle/input/llm-7-prompt-training-dataset/train_essays_RDizzl3_seven_v2.csv')


In [5]:
train_esssays = train_essays_base[['text', 'generated']]

argu = argu_base[['text']]
argu['generated'] = 1

# daigt_1 = daigt_1_base[['text', 'label']]
# daigt_1.rename(columns={'label':'generated'}, inplace=True)
# daigt_2 = daigt_2_base[['text', 'label']]
# daigt_2.rename(columns={'label':'generated'}, inplace=True)
# daigt_3 = daigt_3_base[['text', 'label']]
# daigt_3.rename(columns={'label':'generated'}, inplace=True)
# daigt_4 = daigt_4_base[['text', 'label']]
# daigt_4.rename(columns={'label':'generated'}, inplace=True)

train_ = daigtv2_base[daigtv2_base.RDizzl3_seven == False].reset_index(drop=True)
train_ = daigtv2_base[daigtv2_base["label"]==1].sample(8000)
daigtv2_base = daigtv2_base[daigtv2_base.RDizzl3_seven == True].reset_index(drop=True)
daigtv2_base = pd.concat([daigtv2_base, train_])

daigtv2 = daigtv2_base[['text', 'label']]
daigtv2.rename(columns={'label':'generated'}, inplace=True)


# hello = hello_base[['essay_text']]
# hello.rename(columns={'essay_text':'text'}, inplace=True)
# hello['generated'] = 1

llm = llm_base[['text', 'label']]
llm.rename(columns={'label': 'generated'}, inplace=True)

/tmp/ipykernel_20/1124928610.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  argu['generated'] = 1
/tmp/ipykernel_20/1124928610.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daigtv2.rename(columns={'label':'generated'}, inplace=True)


In [6]:
# df = daigtv2
df = pd.concat([
    train_esssays,
    argu,
    llm,
    daigtv2
])

df['text'] = df['text'].str.replace('\n', ' ')
df['text'] = df['text'].str.replace('\"', '')
df['text'] = df['text'].str.replace('\'', '')
df['text'] = df['text'].str.replace(',', ' ')
df['text'] = df['text'].str.replace('?', ' ')
df['text'] = df['text'].str.replace('!', ' ')
df['text'] = df['text'].str.replace('\\', '')
df['text'] = df['text'].str.replace('\"', '')
df['text'] = df['text'].str.replace(r'<[^>]+>', '', regex=True)
df['text'] = df['text'].str.replace(r'\s+', ' ', regex=True)
df['text'] = df['text'].str.lower()
df['text'] = df['text'].str.replace(f'[{string.punctuation}]', '', regex=True)


In [7]:
df['generated'].value_counts()

generated
0    29872
1    21245
Name: count, dtype: int64

In [8]:
NUMBER_OF_SAMPLES = None
NUMBER_OF_MODELS = None

In [9]:
df = df.dropna()
# negative = df[df['generated'] == 0]
# positive = df[df['generated'] == 1]
# df = pd.concat([negative.sample(NUMBER_OF_SAMPLES), positive.sample(NUMBER_OF_SAMPLES)])
# df = df.sample(NUMBER_OF_SAMPLES)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51117 entries, 0 to 39407
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       51117 non-null  object
 1   generated  51117 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.2+ MB


In [10]:
class VClassifier():

    def __init__(self, number_of_samples, number_of_models):
        self.seed = 2023
        self.seed_everything(self.seed)
        self.tool = language_tool_python.LanguageTool('en-US')
        self.nos = number_of_samples
        self.nom = number_of_models
        self.models = []
        self.voting = None
        self.trained = False
        
    def seed_everything(self, seed=2023):
        import random
        random.seed(seed)
        np.random.seed(seed)

    def correct_sentence(self, sentence):
        return self.tool.correct(sentence)

    def how_many_typos(self, sentence):
        return len(self.tool.check(sentence))

    def insert_df(self, df, x_column, y_column, test):
        self.df = df
        self.x = x_column
        self.y = y_column
        if test is not None:
            self.generate_vectorizer(test)
        else:
            self.generate_vectorizer(df)
#         self.correct_df()

    def correct_df(self, max_workers=2):
        with ProcessPoolExecutor(max_workers=max_workers) as executor:
            self.df[self.x] = self.df[self.x].apply(self.correct_sentence)

    def generate_vectorizer(self, df,  max_feature=None, n_grams=5):
        tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True,
                                             ngram_range=(1, 5),
                                             tokenizer=lambda x: re.findall(r'[^\W]+', x),
                                             token_pattern=None,
                                             strip_accents='unicode',
                                             )
        self.tfidf = tfidf_vectorizer.fit(df[self.x])

    def create_sgd(self, max_iter=5000, tol=1e-3, class_weight=None, early_stopping=False):
        model = SGDClassifier(max_iter=max_iter,
                                  tol=tol,
                                  early_stopping= early_stopping,
                                  class_weight=class_weight,
                                  loss="modified_huber",
                                  verbose=1)
        self.models.append(model)
        print('SGD model creation finished') 
        
    def create_lr(self):
        model = LogisticRegression()
        self.models.append(model)
        print('Logistic Regression model creation finished') 

    def generate_voting(self):
        estimators = []
#         weights = [0.01, 0.99, 0.99, 0.99]
#         self.create_lr()
        self.create_sgd()
        self.create_sgd(class_weight='balanced')
        self.create_sgd(tol=5e-4, max_iter=10000, early_stopping=True)
#         self.create_sgd(class_weight='balanced')
#         self.create_sgd()

        if self.nom is None:
            self.nom = len(self.models)
        
        for idx, model in enumerate(self.models):
            estimators.append(('model' + str(idx), model))

        self.voting = VotingClassifier(estimators=estimators, voting='soft')
        print('Voting generated')
        return self.voting

    def fit_voting(self):
        if self.nos is None:
            self.nos = len(self.df)
        samples = self.df.sample(self.nos)
        train_x, test_x, train_y, test_y = train_test_split(samples[self.x], samples[self.y], test_size=0.2)
        if self.voting:
            train_x = self.tfidf.transform(train_x)
            self.voting.fit(train_x, train_y)
            self.trained = True
            test_x = self.tfidf.transform(test_x)
            predicted = self.voting.predict_proba(test_x)
            # Generate accuracy
#             accuracy = accuracy_score(test_y, predicted[:,1])
#             print(f'Voting Classifier Accuracy: {accuracy}')
        else:
            print('No voting available!')

        print('fitting finished')

    def predict(self, test):
        if self.trained:
            predictions = self.voting.predict_proba(test)
        else:
            print('You must train your models first!')

        return predictions

In [11]:
clf = VClassifier(NUMBER_OF_SAMPLES, NUMBER_OF_MODELS)
clf.insert_df(df, 'text', 'generated', test_essays_base)

In [12]:
clf.generate_voting()

SGD model creation finished
SGD model creation finished
SGD model creation finished
Voting generated


VotingClassifier(estimators=[('model0',
                              SGDClassifier(loss='modified_huber',
                                            max_iter=5000, verbose=1)),
                             ('model1',
                              SGDClassifier(class_weight='balanced',
                                            loss='modified_huber',
                                            max_iter=5000, verbose=1)),
                             ('model2',
                              SGDClassifier(early_stopping=True,
                                            loss='modified_huber',
                                            max_iter=10000, tol=0.0005,
                                            verbose=1))],
                 voting='soft')

In [13]:
clf.fit_voting()

-- Epoch 1
Norm: 1.06, NNZs: 1, Bias: -0.218876, T: 40893, Avg. loss: 0.980133
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 0.96, NNZs: 1, Bias: -0.189893, T: 81786, Avg. loss: 0.971669
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 0.93, NNZs: 1, Bias: -0.107912, T: 122679, Avg. loss: 0.971428
Total training time: 0.01 seconds.
-- Epoch 4
Norm: 0.94, NNZs: 1, Bias: -0.163192, T: 163572, Avg. loss: 0.971185
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 0.93, NNZs: 1, Bias: -0.173270, T: 204465, Avg. loss: 0.970867
Total training time: 0.01 seconds.
-- Epoch 6
Norm: 0.93, NNZs: 1, Bias: -0.181560, T: 245358, Avg. loss: 0.970861
Total training time: 0.01 seconds.
-- Epoch 7
Norm: 0.94, NNZs: 1, Bias: -0.191481, T: 286251, Avg. loss: 0.970829
Total training time: 0.01 seconds.
Convergence after 7 epochs took 0.01 seconds
-- Epoch 1
Norm: 0.91, NNZs: 1, Bias: -0.035391, T: 40893, Avg. loss: 1.008661
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 0.88, NNZs: 1, Bias: 

In [14]:
test_essays = clf.tfidf.transform(test_essays_base['text'])

In [15]:
res = clf.predict(test_essays)

In [16]:
submission = test_essays_base[['id']]
submission['generated'] = res[:,1]

/tmp/ipykernel_20/3496404589.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['generated'] = res[:,1]


In [17]:
submission['generated'] = submission['generated'].apply(lambda a: (int(a)*10 + int(((a%1)*10)))/10)

/tmp/ipykernel_20/3021702422.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['generated'] = submission['generated'].apply(lambda a: (int(a)*10 + int(((a%1)*10)))/10)


In [18]:
submission.to_csv('submission.csv', index=False)